This notebook is designed to demonstrate how to produce plots for airborne EM (line data) and induction (point data) on a vertical section. We compare AEM conductvitiy inversions from a 1D-inversion (stored as netcdf file produced using geophys_utils) and 3D-inversions (stored as asci xyx file).

Neil Symington neil.symington@ga.gov.au

In [6]:
import netCDF4
import os, glob
import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from geophys_utils._netcdf_line_utils import NetCDFLineUtils, NetCDFPointUtils
from geophys_utils._transect_utils import coords2distance
from hydrogeol_utils import SNMR_utils, AEM_utils, spatial_functions, plotting_utils,borehole_utils
import hydrogeol_utils.plotting_utils as plot_utils
import sqlalchemy as db
from sqlite3 import dbapi2 as sqlite
import gc
import sqlalchemy as db
from sqlalchemy import create_engine, event

In [3]:
# First we will deomstrate some straight AEM plotting

# Open netcdf files

#ncdir = r"\\prod.lan\active\proj\futurex\East_Kimberley\Data\Processed\Geophysics\AEM\EK_nbc_inversions\OrdKeep_borehole_constrained\netcdf"
ncdir = r"C:\GA\EK_AEM\inversions\OrdKeep_borehole_constrained\netcdf"
# Open the file with the EM measurements
# Here we use the data response file provided by Niel Christensen
EM_path = os.path.join(ncdir,'OrdKeep2019_DataResp_cor2DLogOrd.nc')
EM_dataset = netCDF4.Dataset(EM_path)


# Open the file with the many layer model conductivity values
# The conductivity model was a 2d correlated borehole
# constrained inversion done by Niel Christensen

cond_path = os.path.join(ncdir,'OrdKeep2019_ModeExp_cor2DLogOrd.nc')
cond_dataset = netCDF4.Dataset(cond_path)




In [4]:
cond_dataset.variables['conductivity']

<class 'netCDF4._netCDF4.Variable'>
float32 conductivity(point, layer)
    _FillValue: 0.0
    long_name: Layer conductivity
    units: S/m
unlimited dimensions: 
current shape = (641851, 30)
filling on

In [5]:
# airborne EM inversions and data are typically visualised on a line by line basis

# To utilise the geophys_utils for line data create a NetCDFLineUtils for each dataset

EM_line_utils = NetCDFLineUtils(EM_dataset)
cond_line_utils = NetCDFLineUtils(cond_dataset)
cond_point_utils = NetCDFPointUtils(cond_dataset)

# Display the lines for the conductivity mode

wkt, aem_coords = cond_point_utils.utm_coords(cond_point_utils.xycoords)

In [32]:
lines = [202702, 202802, 202902, 805900, 806001,
       806101, 806102, 806201, 806301, 806401, 806500, 806601, 806602,
         314301, 314401, 314501, 314601, 314602
        ]

In [34]:
# Create an instance of plots for gridding the data
plots = plot_utils.ConductivitySectionPlot(cond_dataset, EM_dataset)

# Define some key variables which we want to inteprolate

plots.conductivity_variables = ['conductivity', 'data_residual', 'tx_height_measured', 'depth_of_investigation']

plots.EM_variables  = ['data_values_by_low_moment_gate', 'data_values_by_high_moment_gate', 
                       'data_response_by_low_moment_gate', 'data_response_by_high_moment_gate']


# Define the resolution of th sections
xres, yres = 20., 2.

gridded_vars = plots.grid_variables(xres = xres, yres =yres, lines=lines, smoothed = False,
                                    return_dict = True)

In [45]:
def format_panels(axes, panel_settings, plot_settings):
    
    # Iterate through the axes and set the aspect if it is included in the panel settings
    
    grid_ratio = {}
    
    height_ratios = plot_utils.unpack_plot_settings(panel_settings,
                                                    'height_ratio')
    
    for i, ax in enumerate(axes):
        panel = panel_settings['panel_' + str(i + 1)]

        if 'vertical_exaggeration' in panel['panel_kwargs']:
            # Assert it is a grid
            assert panel['plot_type'] == 'grid'
            
            # Get the vertical exageration
            vexag = panel['panel_kwargs']['vertical_exaggeration']
            
            # Set the aspect
            ax.set_aspect(vexag)
            
            # get teh grid ratio
            grid_ratio[i + 1] = (np.diff(ax.get_ylim())/np.diff(ax.get_xlim())) * vexag
    
    ax_pos = {}
    # Iterate through the axes and get position
    for i, ax in enumerate(ax_array):
        # Find the position of each axis
         ax_pos[i] = ax.get_position(original = False)
        
    # Use this to set the size of the graph

    additional_margins   = plot_settings['additional_margins']
    panel_vgap   = plot_settings['panel_vgap']
    plot_width = plot_settings['plot_width']
    
    # Find the height
    
    plot_height = 0.
    plot_height = plot_height + additional_margins + (len(panel_settings) - 1) * panel_vgap
    
    # For gridded items find the axis hgiht
    for item in grid_ratio:
            
        # Get the wdith of the axis in inches
        ax_width = np.min([x.width for x in ax_pos.values()]) * plot_width
        cond_ax_height = ax_width * grid_ratio[item]
        
        plot_height += cond_ax_height
    
    # Now add the height for the other panels
    
    for i in range(len(height_ratios)):
        panel = panel_settings['panel_' + str(i + 1)]
        
        if not 'vertical_exaggeration' in panel['panel_kwargs']:
        
            plot_height += cond_ax_height * (height_ratios[i]/height_ratios[item - 1])
        
    plt.gcf().set_size_inches(plot_width, plot_height)
    

In [46]:
def add_axis_coords(axis_label, array,
                    axis_above, axis_position, offset=-0.15):
    """
    Function for adding a coordinate axis to the bottom of the plot

    :param axis_label:
    :param array:
    :param axis_above:
    :param axis_position:
    :param offset:
    :return:
    """
    new_ax = axis_above.twiny()

    new_ax.set_xlabel(axis_label)

    #new_ax.set_position(axis_position)
    new_ax.xaxis.set_ticks_position("bottom")
    new_ax.xaxis.set_label_position("bottom")
    # Offset the twin axis below the host
    new_ax.spines["bottom"].set_position(("axes", offset))

    # Turn on the frame for the twin axis, but then hide all
    # but the bottom spine
    new_ax.set_frame_on(True)
    new_ax.patch.set_visible(False)

    new_ax.spines["bottom"].set_visible(True)

    # Get tick locations from the old axis

    new_tick_locations = np.array(np.arange(0, 1.1, 0.1))

    new_ax.set_xticks(new_tick_locations)

    # Find the ticks to label

    new_x = plot_utils.griddata(np.linspace(0, 1, num=len(array)), array,
                     new_tick_locations)

    new_ax.set_xticklabels([str(int(x)) for x in new_x])

In [47]:
def plot_grid(ax, gridded_variables, variable, panel_kwargs, x_ax_var = 'grid_distances'):
    """

    :param gridded_variables:
    :param variables:
    :param panel_kwargs:
    :return:
    """

    # Define extents based on kwarg max depth

    try:
        min_elevation = np.min(gridded_variables['elevation']) - panel_kwargs['max_depth']
        ax.set_ylim(min_elevation)

    except KeyError:

        min_elevation = gridded_variables['grid_elevations'][-1]


    extent = (gridded_variables[x_ax_var][0], gridded_variables[x_ax_var][-1],
              gridded_variables['grid_elevations'][-1], gridded_variables['grid_elevations'][0])

    # WE will make the ylim 10% of the depth range

    max_elevation = gridded_variables['grid_elevations'][0] + 0.1 * (gridded_variables['grid_elevations'][0]
                                                                     - min_elevation)

    ax.set_ylim(min_elevation, max_elevation)

    # Define stretch
    # Flag for a logarithmic stretch

    try:
        log_stretch = panel_kwargs['log_plot']

    except KeyError:
        log_stretch = False  # False unless otherwise specified


    if log_stretch:
        # Tranform the plot data
        data = np.log10(gridded_variables[variable])

    else:
        data = gridded_variables[variable]
        # set automatic stretch values in case vmin and vmax aren't specified
        vmin, vmax = 0, 0.5

    # Define vmin an vmax if specified
    if 'vmin' in panel_kwargs.keys():
        vmin = panel_kwargs['vmin']
    if 'vmax' in panel_kwargs.keys():
        vmax = panel_kwargs['vmax']

    if log_stretch:
        vmin, vmax = np.log10(vmin), np.log10(vmax)

    # Define cmap if it is specified
    if 'cmap' in panel_kwargs.keys():
        cmap = panel_kwargs['cmap']

    else:
        cmap = 'jet'

    # Plot data

    im = ax.imshow(data, vmin=vmin, vmax=vmax,
                       extent=extent,
                       aspect='auto',
                       cmap=cmap)

    # Plot the elevation as a line over the section
    line_x = np.linspace(gridded_variables[x_ax_var][0], gridded_variables[x_ax_var][-1],
                         np.shape(gridded_variables[variable])[1])

    ax.plot(line_x, gridded_variables['elevation'], 'k')

    # To remove gridded values that stick above this line we will fill the sky in as white
    ax.fill_between(line_x, max_elevation * np.ones(np.shape(line_x)),
                    gridded_variables['elevation'], interpolate=True, color='white', alpha=1)

    # Add ylabel
    try:
        ylabel = panel_kwargs['ylabel']
        ax.set_ylabel(ylabel)
    except KeyError:
        pass

    # PLot depth of investigation and make area underneath more transparent if desired
    if panel_kwargs['shade_doi']:
        eoi = gridded_variables['elevation'] - gridded_variables['depth_of_investigation']

        ax.plot(line_x, eoi, 'k')

        grid_base = gridded_variables['grid_elevations'][-1]

        # Shade the belwo doi areas

        ax.fill_between(line_x, eoi, grid_base, interpolate=True, color='white', alpha=0.5)
        
    return im
        
def plot_single_line(ax, gridded_variables, variable, panel_kwargs):
    """

    :param ax:
    :param gridded_variables:
    :param variables:
    :param panel_kwargs:
    :return:
    """
    # Define the array

    data = gridded_variables[variable]

    if 'colour' in panel_kwargs.keys():
        colour = panel_kwargs['colour']
    else:
        colour = 'black'


    lin = ax.plot(gridded_variables['grid_distances'], data, colour)

    # Extract ymin and ymax if specified, otherwise assign based on the range with the line dataset
    if 'ymin' in panel_kwargs.keys():
        ymin = panel_kwargs['ymin']
    else:
        ymin = np.min(data) - 0.1 * np.min(data)

    if 'ymax' in panel_kwargs.keys():
        ymax = panel_kwargs['ymax']
    else:
        ymax = np.max(data) - 0.1 * np.max(data)

    ax.set_ylim(bottom=ymin, top=ymax, auto=False)

    try:
        ylabel  = panel_kwargs['ylabel']
        ax.set_ylabel(ylabel)
    except KeyError:
        pass

    try:
        if panel_kwargs['legend']:
            ax.legend()
    except KeyError:
        pass
    
    return lin

def plot_multilines_data(ax, gridded_variables, variable, panel_kwargs):
    # Define the data

    data = gridded_variables[variable]

    try:
        colour = panel_kwargs["colour"]
        linewidth = panel_kwargs["linewidth"]
    except KeyError:
        colour = 'k'
        linewidth = 1
    lins = []
    for i, col in enumerate(data.T):
        lin = ax.plot(gridded_variables['grid_distances'], data.T[i],
                color = colour, linewidth = linewidth)
        lins.append(lin)
        ax.set_yscale('log')
    try:
        ylabel = panel_kwargs['ylabel']
        ax.set_ylabel(ylabel)
    except KeyError:
        pass
    
    return lins

def plot_conductivity_section(ax_array, gridded_variables, plot_settings, panel_settings,
                              save_fig=False,  outfile=None):
    """

    :param gridded_variables:
    :param plot_settings:
    :param panel_settings:
    :param save_fig:
    :param outfile:
    :return:
    """

    # Unpack the panel settings

    variables = plot_utils.unpack_plot_settings(panel_settings,
                                          'variable')

    panel_kwargs = plot_utils.unpack_plot_settings(panel_settings,
                                             'panel_kwargs')

    plot_type = plot_utils.unpack_plot_settings(panel_settings,
                                          'plot_type')

    plot_objs = []
    # Iterate through the axes and plot
    for i, ax in enumerate(ax_array):
        
        # Create an axis divider

        if 'title' in panel_kwargs:
            ax.set_title(panel_kwargs['title'])
        else:
            ax.set_title(' '.join([variables[i].replace('_', ' '), 'plot']))

        if plot_type[i] == 'grid':

            # PLot the grid
            plot_objs.append(plot_grid(ax, gridded_variables,
                                       variables[i], panel_kwargs[i]))

        elif plot_type[i] == 'multi_line':

            plot_objs.append(plot_utils.plot_multilines_data(ax, gridded_variables,
                                                             variables[i], panel_kwargs[i]))

        elif plot_type[i] == 'line':
            plot_objs.append(plot_utils.plot_single_line(ax, gridded_variables, variables[i],
                                                         panel_kwargs[i]))

    return plot_objs

    
def add_colourbar(fig, ax, im, x0, y0, width, height, panel_kwargs):
    
    # Define the colourmap based on the panel kwargs input
    if 'colourbar_label' in panel_kwargs.keys(): 
        cm = plt.cm.get_cmap(panel_kwargs['cmap'])
    else:
        cm = plt.cm.get_cmap('jet')
    
    # Transform rel position and transform them after to get the 
    # ABSOLUTE POSITION AND DIMENSIONS
    Bbox = mpl.transforms.Bbox.from_bounds(x0, y0, width, height)
    trans = ax.transAxes + fig.transFigure.inverted()
    l, b, w, h = mpl.transforms.TransformedBbox(Bbox, trans).bounds

    # Now just create the axes and the colorbar
    cbaxes = fig.add_axes([l, b, w, h])
    cb = plt.colorbar(im, cax=cbaxes,
                      orientation='vertical')
    # If logplot then transform the labels bac to S/m
    if panel_kwargs['log_plot']:
        cb.ax.set_yticklabels([round(10 ** x, 4) for x in cb.get_ticks()])
    
    if 'colourbar_label' in panel_kwargs.keys():
        cb.ax.tick_params(labelsize=9)
        cb_label = panel_kwargs['colourbar_label']
        cb.set_label(cb_label, fontsize=10)
        cb.ax.tick_params(labelsize=10)
    

In [49]:
# Now lets plot with the data over the top


panel_settings = {'panel_1': {'variable': 'data_values_by_high_moment_gate',
                              'plot_type': 'multi_line',
                             'panel_kwargs': {'title': 'high moment data',
                                             'ylabel': 'dB/dT (V/(A.turns.m^4))'},
                             'height_ratio': 2},
                  
                 'panel_2': {'variable': 'data_values_by_low_moment_gate',
                             'plot_type': 'multi_line',
                             'panel_kwargs': {'title': 'low moment data',
                                              'ylabel': 'dB/dT (V/(A.turns.m^4))'},
                             'height_ratio': 2},
                  
                 'panel_3': {'variable': 'data_residual',
                             'plot_type': 'line',
                             'panel_kwargs': {'title': 'data residual', 'color': 'black',
                                              'ylabel': 'data residual',
                                              'legend': False},
                             'height_ratio': 0.5},
                  
                 'panel_4': {'variable': 'conductivity',
                             'plot_type': 'grid',
                             'panel_kwargs': {'title': 'AEM conductivity',
                                              'max_depth': 200, 'shade_doi': True, 'colourbar': True,
                                              'colourbar_label': 'Conductivity (S/m)',
                                             'log_plot': True, 'vmin': 0.001, 'vmax': 1,
                                             'cmap': 'jet', 'ylabel': 'elevation_(mAHD)',
                                             'vertical_exaggeration': 10.},
                             'height_ratio': 2}}



height_ratios = plot_utils.unpack_plot_settings(panel_settings,'height_ratio')


plot_settings = {'additional_margins': 2.5,
                    'panel_vgap': 1.5, 'plot_width': 11.7 #A3 width
                }


outdir = r"C:\temp\sections"

panel_kwargs = plot_utils.unpack_plot_settings(panel_settings,
                                             'panel_kwargs')

for line in lines:
    
    plt.close()

    fig, ax_array = plt.subplots(len(panel_settings),1, sharex = True,
                                gridspec_kw={'height_ratios': height_ratios,
                                            'wspace':plot_settings['panel_vgap']})

    plot_objs = plot_conductivity_section(ax_array, gridded_vars[line], plot_settings,
                                         panel_settings, save_fig = False, )
     
    
    # Set the aspect based on the vertical exageration
    
    format_panels(ax_array, panel_settings, plot_settings)
            
    # Tighten the figure    
    
    fig.tight_layout()
            
    # Relative position of the colourbar
    x0, y0, width, height = [1.01, 0., 0.02, 1.
                            ]
    # Now we want to add the colorbar and coordinates axes
    for i, ax in enumerate(ax_array):
        try:
            if panel_kwargs[i]['colourbar']:
        
                # Add the colour bar to the plots that the user specifies
                add_colourbar(fig, ax_array[i], plot_objs[i], x0, y0, width, height, panel_kwargs[i])
        except KeyError:
            pass
        
    # Add coordinate axes to the bottom of the plot
    ax_pos = plot_utils.align_axes(ax_array)
    
    add_axis_coords('northing', gridded_vars[line]['northing'],
                               ax_array[-1], ax_pos[len(panel_settings) - 1], offset=-0.15)

    add_axis_coords('easting', gridded_vars[line]['easting'], ax_array[-1],
                               ax_pos[len(panel_settings) - 1], offset=-0.4)
    
    outfile = os.path.join(outdir, str(line) + '.png')
    plt.savefig(outfile)
    
    #plt.show()
    
    gc.collect()

C:\Users\symin\Anaconda3\envs\hydrogeol_utils\lib\site-packages\matplotlib\figure.py:2022: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


In [22]:
panel_kwargs[i]['colourbar_label']

'Conductivity (S/m)'